# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
import torch
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from sklearn.model_selection import RepeatedStratifiedKFold
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from pytorch_tabular.utils import get_balanced_sampler


# Load data

In [ ]:
path_data = "D:/YandexDisk/Work/bbd/immunology/002_central_vs_yakutia/classification"
path_configs = "D:/Work/bbs/notebooks/immunology/002_central_vs_yakutia/pt_configs"
data = pd.read_excel(f"{path_data}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path_data}/feats.xlsx", index_col=0).index.values.tolist()

test_split_id = 0

val_n_splits = 4
val_random_state = 1337
val_fold_id = 0

for fold_id in range(val_n_splits):
    data[f"Fold_{fold_id}"] = data[f"Split_{test_split_id}"]

stratify_cat_parts = {
    'Central': data.index[(data['Region'] == 'Central') & (data[f"Split_{test_split_id}"] == 'trn_val')].values,
    'Yakutia': data.index[(data['Region'] == 'Yakutia') & (data[f"Split_{test_split_id}"] == 'trn_val')].values,
}
for part, ids in stratify_cat_parts.items():
    print(f"{part}: {len(ids)}")
    con = data.loc[ids, 'Age'].values
    ptp = np.ptp(con)
    num_bins = 5
    bins = np.linspace(np.min(con) - 0.1 * ptp, np.max(con) + 0.1 * ptp, num_bins + 1)
    binned = np.digitize(con, bins) - 1
    unique, counts = np.unique(binned, return_counts=True)
    occ = dict(zip(unique, counts))
    k_fold = RepeatedStratifiedKFold(
        n_splits=val_n_splits,
        n_repeats=1,
        random_state=val_random_state
    )
    splits = k_fold.split(X=ids, y=binned, groups=binned)
    
    for fold_id, (ids_trn, ids_val) in enumerate(splits):
        data.loc[ids[ids_trn], f"Fold_{fold_id}"] = "trn"
        data.loc[ids[ids_val], f"Fold_{fold_id}"] = "val"
        
test = data.loc[data[f"Split_{test_split_id}"] == "tst", feats + ['Region']]
train_validation = data.loc[data[f"Split_{test_split_id}"] == "trn_val", feats + ['Region'] + [f"Fold_{i}" for i in range(val_n_splits)]]
train_only = data.loc[data[f"Fold_{val_fold_id}"] == "trn", feats + ['Region']]
validation_only = data.loc[data[f"Fold_{val_fold_id}"] == "val", feats + ['Region']]
cv_indexes = [
    (
        np.where(train_validation.index.isin(train_validation.index[train_validation[f"Fold_{i}"] == 'trn']))[0],
        np.where(train_validation.index.isin(train_validation.index[train_validation[f"Fold_{i}"] == 'val']))[0],
    )
    for i in range(val_n_splits)
]

## Prepare balanced sampler

In [ ]:
sampler_balanced = get_balanced_sampler(train_only['Region'].values.ravel())

# Models Search Spaces

## CategoryEmbeddingModel Search Space

In [ ]:
search_space = {
    "model_config__layers": ["256-128-64", "512-256-128", "32-16", "32-32-16", "16-8", "32-16-8", "128-64", "128-128", "16-16"],
    "model_config.head_config__dropout": [0.0, 0.05, 0.1, 0.15, 0.2, 0.25],
    "model_config__learning_rate": [0.001],
    "model_config__seed": [42, 1337, 666],
}
model_config = read_parse_config(f"{path_configs}/models/CategoryEmbeddingModelConfig.yaml", CategoryEmbeddingModelConfig)
grid_size = np.prod([len(p_vals) for _, p_vals in search_space.items()])
print(grid_size)

## GANDALF Search Space

In [ ]:
search_space = {
    "model_config__gflu_stages": [5, 10, 15, 20, 25, 30, 35],
    "model_config__gflu_dropout": [0.0, 0.05, 0.1, 0.15, 0.2, 0.25],
    "model_config__gflu_feature_init_sparsity": [0.1, 0.2, 0.3, 0.4, 0.5],
    "model_config.head_config__dropout": [0.0, 0.05, 0.1, 0.15, 0.2, 0.25],
    "model_config__learning_rate": [0.001],
    "model_config__seed": [1337, 666],
}
model_config = read_parse_config(f"{path_configs}/models/GANDALFConfig.yaml", GANDALFConfig)
grid_size = np.prod([len(p_vals) for _, p_vals in search_space.items()])
print(grid_size)

# Grid Search and Random Search

In [ ]:
%%capture

strategy = 'random_search' # 'grid_search'
seed = 1337
n_random_trials = 200
is_cross_validation = True

if grid_size < n_random_trials and strategy == 'random_search':
    strategy = 'grid_search'

data_config = read_parse_config(f"{path_configs}/DataConfig.yaml", DataConfig)
data_config['continuous_feature_transform'] = 'yeo-johnson'
data_config['normalize_continuous_features'] = True
trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
trainer_config['checkpoints'] = None
trainer_config['load_best'] = False
trainer_config['auto_lr_find'] = True
optimizer_config = read_parse_config(f"{path_configs}/OptimizerConfig.yaml", OptimizerConfig)

tuner = TabularModelTuner(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    suppress_lightning_logger=True,
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    if is_cross_validation:
        result = tuner.tune(
            train=train_validation,
            validation=None,
            search_space=search_space,
            metric="accuracy",
            mode="max",
            strategy=strategy,
            n_trials=n_random_trials,
            cv=cv_indexes,
            return_best_model=True,
            verbose=False,
            progress_bar=False,
            random_state=seed,
            train_sampler=sampler_balanced
        )
    else:
        result = tuner.tune(
            train=train_only,
            validation=validation_only,
            search_space=search_space,
            metric="accuracy",
            mode="max",
            strategy=strategy,
            n_trials=n_random_trials,
            cv=None,
            return_best_model=True,
            verbose=False,
            progress_bar=False,
            random_state=seed,
            train_sampler=sampler_balanced
        )

result.trials_df.to_excel(f"{trainer_config['checkpoints_path']}/trials/{model_config['_model_name']}_{strategy}_{seed}_{optimizer_config['lr_scheduler']}.xlsx")

# Simple TabularModel training

In [ ]:
trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
trainer_config['checkpoints'] = 'valid_loss'
trainer_config['load_best'] = True
trainer_config['auto_lr_find'] = True

tabular_model = TabularModel(
    data_config=f"{path_configs}/DataConfig.yaml",
    model_config=f"{path_configs}/models/CategoryEmbeddingModelConfig.yaml",
    optimizer_config=f"{path_configs}/OptimizerConfig.yaml",
    trainer_config=trainer_config,
    verbose=True,
    suppress_lightning_logger=False
)

tabular_model.fit(
    train=train_only,
    validation=validation_only,
    # target_transform=[np.log, np.exp],
    # callbacks=[DeviceStatsMonitor()],
)

## Play with trained model

In [ ]:
tabular_model.predict(test, progress_bar='rich')

In [ ]:
tabular_model.evaluate(test, verbose=True, ckpt_path="best")

In [ ]:
tabular_model.config['checkpoints_path']

In [ ]:
print(tabular_model.trainer.checkpoint_callback.best_model_path)

In [ ]:
tabular_model.summary()

In [ ]:
tabular_model.save_model(tabular_model.config['checkpoints_path'])

In [ ]:
tabular_model.save_config(tabular_model.config['checkpoints_path'])

In [ ]:
tabular_model = TabularModel.load_model(tabular_model.config['checkpoints_path'])